In [35]:


import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Para modelos binarios y Croston:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, mean_absolute_error
from darts import TimeSeries
from darts.models import Croston

In [36]:
# 1. LECTURA DE DATOS

data_path = 'db.csv'  #Ruta
df = pd.read_csv(f'{data_path}', sep=',')

print(df.info())
display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64765 entries, 0 to 64764
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Billing date       64765 non-null  int64 
 1   Material           64765 non-null  int64 
 2   Phisical quantity  64765 non-null  int64 
 3   Country            64765 non-null  int64 
 4   Net price          64765 non-null  object
dtypes: int64(4), object(1)
memory usage: 2.5+ MB
None


,Billing date,Material,Phisical quantity,Country,Net price
0,10052022,1795,10,7511,"2,865"
1,10052022,1778,5,7511,"0,66375"
2,10052022,1778,15,7511,"0,66375"
3,10052022,9526,4,7511,"36,31125"
4,10052022,1778,11,7511,"0,66375"
5,10052022,1778,15,7511,"0,66375"
6,10052022,1778,5,7511,"0,66375"
7,10052022,2844,2,7511,"35,185"
8,10052022,4151,6,7511,"31,47875"
9,10052022,2844,2,7511,"35,185"


In [37]:
# 2. PREPROCESAMIENTO INICIAL


def process_date_str(number):
    """
    Convierte un int tipo ddmmaaaa (ej: 10052022) a 'yyyy-mm-dd'.
    """
    string = str(number)
    if len(string) == 7:
        # Caso en que falta un dígito
        string = '0' + string
    # Extrae (dd, mm, aaaa) => ([:2], [2:4], [4:])
    # y formatea a 'aaaa-mm-dd'
    return f'{string[4:]}-{string[2:4]}-{string[:2]}'

df['time_str'] = df['Billing date'].apply(process_date_str)
df['time'] = pd.to_datetime(df['time_str'], format='%Y-%m-%d')

# Se ajusta el lunes como dia inicial (opcional)
df['time'] = df['time'] - pd.to_timedelta(df['time'].dt.weekday, unit='D')

# Limpiamos Net price (reemplazando comas por puntos)
df['Net price'] = df['Net price'].apply(lambda x: x.replace(',', '.')).astype(float)

In [38]:
# 3. AGRUPACIÓN DIARIA + PREPARACIÓN

grouped_df = df.groupby(['Material', 'time']).sum().reset_index()[
    ['time', 'Material', 'Phisical quantity', 'Net price']
]
grouped_df = grouped_df.rename(columns={
    'Material': 'item',
    'Phisical quantity': 'sales',
    'Net price': 'price'
})
grouped_df = grouped_df.sort_values(by='time')


def fill_no_sales_period(df, item_id):
    """
    Rellena con filas (sales=0) aquellos días que no existen en df para el item_id.
    """
    item_df = df[df['item'] == item_id]
    existing_dates = item_df['time'].to_list()
    if len(item_df) == 0:
        return df  # Si no hay datos de ese item, retornamos como está

    first_sale_date = item_df['time'].min().strftime("%Y-%m-%d")
    last_sale_date = item_df['time'].max().strftime("%Y-%m-%d")

    # Frecuencia diaria ('D'). 
    time_index = pd.date_range(start=first_sale_date, end=last_sale_date, freq='D')

    new_data = {'time': [], 'item': [], 'sales': [], 'price': []}
    for date in time_index:
        if date not in existing_dates:
            new_data['time'].append(date)
            new_data['item'].append(item_id)
            new_data['sales'].append(0)
            new_data['price'].append(0.0)

    new_dates_df = pd.DataFrame(new_data)
    df = pd.concat([df, new_dates_df], axis=0)
    return df

items = list(grouped_df['item'].unique())
full_daily_df = grouped_df.copy()

for item in items:
    full_daily_df = fill_no_sales_period(full_daily_df, item)

full_daily_df['sales'] = full_daily_df['sales'].astype(int)
full_daily_df['item'] = full_daily_df['item'].astype(int)

/var/folders/5x/tlp5fd614dv9y_wn8nmtqv4rqq_frn/T/ipykernel_40886/912565248.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, new_dates_df], axis=0)


In [39]:
full_daily_df

,time,item,sales,price
12646,2018-08-20,3531,2,13.05125
26002,2018-08-20,6488,2,14.11000
1765,2018-08-20,1435,3,6.47625
41804,2018-08-20,9899,30,1.11375
5929,2018-08-20,2210,3,5.84125
...,...,...,...,...
8,2024-08-21,1978,0,0.00000
9,2024-08-22,1978,0,0.00000
10,2024-08-23,1978,0,0.00000
11,2024-08-24,1978,0,0.00000


In [40]:
# 4. CLASIFICACIÓN DE DEMANDA (p y CV2)

def calculate_p_interval(df, item):
    filtered_df = df[df['item'] == item]
    total_periods = float(len(filtered_df))
    non_zero_demands = float(len(filtered_df[filtered_df['sales'] > 0]))
    if non_zero_demands == 0:
        return np.inf
    return total_periods / non_zero_demands

def calculate_CV2(df, item):
    filtered_df = df[(df['item'] == item) & (df['sales'] > 0)]
    if len(filtered_df) < 2:
        return 0
    std = np.std(filtered_df['sales'])
    mean_ = np.mean(filtered_df['sales'])
    if mean_ == 0:
        return 0
    return (std / mean_) ** 2

demand_factors = []
for it in items:
    p_val = calculate_p_interval(full_daily_df, it)
    cv2_val = calculate_CV2(full_daily_df, it)
    demand_factors.append((it, p_val, cv2_val))

classification_df = pd.DataFrame(demand_factors, columns=['item', 'p', 'cv2'])

def classify_demand(row):
    THRESHOLD_P = 1.32
    THRESHOLD_CV2 = 0.49
    p = row['p']
    cv2 = row['cv2']
    if p < THRESHOLD_P and cv2 >= THRESHOLD_CV2:
        return 'erratic'
    elif p >= THRESHOLD_P and cv2 >= THRESHOLD_CV2:
        return 'lumpy'
    elif p < THRESHOLD_P and cv2 < THRESHOLD_CV2:
        return 'smooth'
    else:
        return 'intermittent'

classification_df['demand_type'] = classification_df.apply(classify_demand, axis=1)

# Precio unitario promedio
full_daily_df['price'] = full_daily_df['price'].fillna(0.0)
full_daily_df['unit_price'] = np.where(
    full_daily_df['sales'] > 0,
    full_daily_df['price'] / full_daily_df['sales'],
    0
)
filtered_nonzero = full_daily_df[full_daily_df.sales > 0]
avg_price_df = filtered_nonzero[['item', 'unit_price']].groupby('item').mean().reset_index()
avg_price_df.rename(columns={'unit_price': 'avg_unit_price'}, inplace=True)

item_df = classification_df.merge(avg_price_df, on='item', how='left')

In [41]:
# 5. ENTRENAMIENTO Y COMPARACIÓN (TODOS LOS ÍTEMS)

results_list = []
all_daily_preds_list = []  # Para luego generar predicciones mensuales

item_ids = classification_df['item'].unique()

for it in item_ids:
    item_data = full_daily_df[full_daily_df.item == it].sort_values('time').reset_index(drop=True)

    # Si hay pocos datos, saltamos
    if len(item_data) < 10:
        results_list.append({
            'item': it,
            'demand_type': classification_df.loc[classification_df['item']==it, 'demand_type'].values[0],
            'n_points': len(item_data),
            'bin_reg_mae': None,
            'croston_mae': None,
            'croston_est_mae': None
        })
        continue


    # (A) MODELO BINARIO + REGRESIÓN

    item_data['has_sales'] = (item_data['sales'] > 0).astype(int)
    item_data['month_idx'] = item_data['time'].dt.month
    item_data['dow'] = item_data['time'].dt.weekday  # Podrías usarlo si quisieras
    item_data['lag1'] = item_data['sales'].shift(1).fillna(0)
    item_data['lag2'] = item_data['sales'].shift(2).fillna(0)

    train_size = int(len(item_data)*0.8)
    train_df = item_data.iloc[:train_size].copy()
    test_df  = item_data.iloc[train_size:].copy()

    if len(train_df)==0 or len(test_df)==0:
        results_list.append({
            'item': it,
            'demand_type': classification_df.loc[classification_df['item']==it, 'demand_type'].values[0],
            'n_points': len(item_data),
            'bin_reg_mae': None,
            'croston_mae': None,
            'croston_est_mae': None
        })
        continue

    X_train = train_df[['month_idx', 'lag1', 'lag2']]  # Ejemplo: mes del año + lags
    y_train = train_df['has_sales']
    X_test  = test_df[['month_idx', 'lag1', 'lag2']]
    y_test  = test_df['has_sales']

    clf = LogisticRegression()
    try:
        clf.fit(X_train, y_train)
    except:
        results_list.append({
            'item': it,
            'demand_type': classification_df.loc[classification_df['item']==it, 'demand_type'].values[0],
            'n_points': len(item_data),
            'bin_reg_mae': None,
            'croston_mae': None,
            'croston_est_mae': None
        })
        continue

    # Predicción binaria
    test_df['pred_has_sales'] = clf.predict(X_test)
    test_df['pred_sales_bin'] = 0.0

    # Regresión para la magnitud
    reg = LinearRegression()
    train_df_reg = train_df[train_df['sales']>0].copy()
    bin_reg_mae = None
    if len(train_df_reg)>0:
        X_train_reg = train_df_reg[['month_idx', 'lag1', 'lag2']]
        y_train_reg = train_df_reg['sales']

        try:
            reg.fit(X_train_reg, y_train_reg)
        except:
            pass

        idx_has_sales = test_df.index[test_df['pred_has_sales']==1]
        X_test_reg = test_df.loc[idx_has_sales, ['month_idx', 'lag1', 'lag2']]
        if not X_test_reg.empty:
            y_pred_reg = reg.predict(X_test_reg)
            test_df.loc[idx_has_sales, 'pred_sales_bin'] = y_pred_reg

        bin_reg_mae = mean_absolute_error(test_df['sales'], test_df['pred_sales_bin'])


    # (B) CROSTON PURO

    item_ts = TimeSeries.from_dataframe(
        item_data, time_col='time', value_cols='sales', freq='D'
    )
    train_ts = item_ts[:train_size]
    test_ts  = item_ts[train_size:]

    croston_model = Croston()
    try:
        croston_model.fit(train_ts)
        croston_pred = croston_model.predict(len(test_ts))
        croston_values = croston_pred.values().flatten()
        test_values = test_ts.values().flatten()
        croston_mae_val = np.mean(np.abs(test_values - croston_values))
    except:
        croston_values = [np.nan]*len(test_df)
        croston_mae_val = None

    # Pasar predicciones de Croston a DataFrame para merge
    croston_pred_df = pd.DataFrame({
        'time': test_df['time'],
        'pred_croston': croston_values
    })


    # (C) CROSTON + ESTACIONALIDAD (MES del AÑO)

    # 1. Calculamos factor estacional por mes del año
    #    (ejemplo para tomar: moy=1..12 => factor = mean(sales en ese mes) / mean(sales global))
    item_data['moy'] = item_data['time'].dt.month
    global_mean = item_data['sales'].mean() if item_data['sales'].mean() != 0 else 1e-6
    season_factor = item_data.groupby('moy')['sales'].mean() / global_mean

    def apply_seasonal_adjustment(row):
        if row['sales'] == 0:
            return 0
        return row['sales'] / season_factor.loc[row['moy']]

    item_data['sales_adj'] = item_data.apply(apply_seasonal_adjustment, axis=1)

    # 2. Entrenar Croston sobre 'sales_adj'
    item_ts_adj = TimeSeries.from_dataframe(
        item_data, time_col='time', value_cols='sales_adj', freq='D'
    )
    train_ts_adj = item_ts_adj[:train_size]
    test_ts_adj  = item_ts_adj[train_size:]

    croston_model_adj = Croston()
    try:
        croston_model_adj.fit(train_ts_adj)
        croston_pred_adj = croston_model_adj.predict(len(test_ts_adj))
        adj_values = croston_pred_adj.values().flatten()

        # Revertir la estacionalidad
        test_df_adj = item_data.iloc[train_size:].copy()
        test_df_adj['pred_croston_est'] = adj_values

        def revert_season_factor(row):
            return row['pred_croston_est'] * season_factor.loc[row['moy']]

        test_df_adj['pred_croston_est'] = test_df_adj.apply(revert_season_factor, axis=1)

        croston_est_mae_val = mean_absolute_error(
            test_df_adj['sales'],
            test_df_adj['pred_croston_est']
        )
    except:
        croston_est_mae_val = None
        test_df_adj = test_df.copy()
        test_df_adj['pred_croston_est'] = np.nan

    # DataFrame con predicción "estacional" para merge
    croston_est_pred_df = test_df_adj[['time', 'pred_croston_est']].copy()


    # (D) Unimos las predicciones diarias (test set)
    #     - Real, binario, croston, croston_est

    test_pred_df = test_df[['time','sales','pred_sales_bin']].merge(
        croston_pred_df, on='time', how='left'
    ).merge(
        croston_est_pred_df, on='time', how='left'
    )
    test_pred_df['item'] = it

    all_daily_preds_list.append(test_pred_df)


    # (E) Guardamos las métricas globales

    results_list.append({
        'item': it,
        'demand_type': classification_df.loc[classification_df['item']==it, 'demand_type'].values[0],
        'n_points': len(item_data),
        'bin_reg_mae': bin_reg_mae,
        'croston_mae': croston_mae_val,
        'croston_est_mae': croston_est_mae_val
    })

In [42]:
# 6. METRICAS FINALES POR ÍTEM

results_df = pd.DataFrame(results_list)
results_df.sort_values(by='item', inplace=True)
results_df.reset_index(drop=True, inplace=True)

display(results_df.head(50))
# results_df.to_csv('comparacion_croston_mes_ano.csv', sep=';', index=False)

,item,demand_type,n_points,bin_reg_mae,croston_mae,croston_est_mae
0,1002,lumpy,1793,0.222841,1.274647,1.348773
1,1003,intermittent,1170,0.008547,0.040301,0.025591
2,1006,lumpy,1947,0.079487,0.382824,0.337329
3,1007,smooth,1,NaN,NaN,NaN
4,1009,smooth,1,NaN,NaN,NaN
5,1013,intermittent,197,3.750000,196.250000,63.103448
6,1014,lumpy,1982,0.251889,0.700295,0.630759
7,1018,lumpy,694,0.431655,0.665562,0.589222
8,1019,lumpy,2094,0.541766,3.001722,4.044760
9,1020,lumpy,1247,0.340000,0.582923,0.515767


In [43]:
results_df.croston_mae.mean()

5.164536597504412

In [44]:
results_df.croston_est_mae.mean()

2.6029063728845196

In [45]:
# 7. GENERAR Y EXPORTAR PREDICCIONES MENSUALES

all_daily_preds_df = pd.concat(all_daily_preds_list, ignore_index=True)
all_daily_preds_df['time'] = pd.to_datetime(all_daily_preds_df['time'])
all_daily_preds_df.set_index('time', inplace=True)

# Agrupamos por (item, mes) sumando
monthly_preds = (
    all_daily_preds_df
    .groupby(['item', pd.Grouper(freq='M')])
    .agg({
        'sales': 'sum',
        'pred_sales_bin': 'sum',
        'pred_croston': 'sum',
        'pred_croston_est': 'sum'
    })
    .reset_index()
    .rename(columns={
        'time': 'month',
        'sales': 'y_real',
        'pred_sales_bin': 'y_binario',
        'pred_croston': 'y_croston',
        'pred_croston_est': 'y_croston_est'
    })
    .sort_values(by=['item','month'])
    .reset_index(drop=True)
)

display(monthly_preds.head(50))
monthly_preds.to_excel('predicciones_mensuales_croston_est_mes.xlsx', index=False)
print("Archivo 'predicciones_mensuales_croston_est_mes.xlsx' generado con éxito.")


/var/folders/5x/tlp5fd614dv9y_wn8nmtqv4rqq_frn/T/ipykernel_40886/1517329541.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .groupby(['item', pd.Grouper(freq='M')])


,item,month,y_real,y_binario,y_croston,y_croston_est
0,1002,2023-05-31,0,0.0,4.254629,7.539547
1,1002,2023-06-30,0,0.0,31.909721,9.159404
2,1002,2023-07-31,0,0.0,32.973379,0.000000
3,1002,2023-08-31,0,0.0,32.973379,72.379647
4,1002,2023-09-30,0,0.0,31.909721,7.754962
5,1002,2023-10-31,0,0.0,32.973379,64.624685
6,1002,2023-11-30,0,0.0,31.909721,18.094912
7,1002,2023-12-31,40,0.0,32.973379,12.924937
8,1002,2024-01-31,0,0.0,32.973379,51.699748
9,1002,2024-02-29,0,0.0,30.846064,13.197999


Archivo 'predicciones_mensuales_croston_est_mes.xlsx' generado con éxito.


In [46]:
monthly_preds

,item,month,y_real,y_binario,y_croston,y_croston_est
0,1002,2023-05-31,0,0.0,4.254629,7.539547
1,1002,2023-06-30,0,0.0,31.909721,9.159404
2,1002,2023-07-31,0,0.0,32.973379,0.000000
3,1002,2023-08-31,0,0.0,32.973379,72.379647
4,1002,2023-09-30,0,0.0,31.909721,7.754962
...,...,...,...,...,...,...
24317,9998,2023-07-31,0,0.0,40.323716,77.085291
24318,9998,2023-08-31,2,0.0,40.323716,22.201770
24319,9998,2023-09-30,0,0.0,39.022951,39.427280
24320,9998,2023-10-31,0,0.0,40.323716,0.765578


### Aplicamos la métrica custom de Utilidad que definimos anteriormente

In [47]:
classification_df = pd.read_csv('demand_classification_by_item.csv', sep=';')
len(classification_df.item.unique())

3817

In [48]:
classification_df

,item,p,cv2,demand_type,unit_price
0,1002,128.071429,0.612206,lumpy,0.009178
1,1003,234.000000,0.213018,intermittent,1.005850
2,1006,84.652174,4.362716,lumpy,2.061909
3,1007,1.000000,0.000000,smooth,0.398125
4,1009,1.000000,0.000000,smooth,0.133375
...,...,...,...,...,...
3812,9988,312.000000,0.000000,intermittent,1.943750
3813,9989,1.000000,0.000000,smooth,0.191875
3814,9994,150.000000,0.291320,intermittent,4.370863
3815,9997,1.000000,0.000000,smooth,0.106354


In [49]:
classification_dict = classification_df.to_dict()
item_price = {}
for i in classification_dict['unit_price'].keys():
    item_price[classification_dict['item'][i]] = classification_dict['unit_price'][i]

item_price

{1002: 0.009177519132653,
 1003: 1.00585,
 1006: 2.061908695652174,
 1007: 0.398125,
 1009: 0.133375,
 1013: 0.005403125,
 1014: 0.0154515029761904,
 1018: 0.1462920673076923,
 1019: 0.1056544325017709,
 1020: 5.4347615740740745,
 1024: 1.9328125,
 1028: 0.1854576822916667,
 1030: 0.91075,
 1034: 9.77227356362773,
 1035: 0.4789583333333333,
 1037: 4.2075,
 1040: 5.03875,
 1044: 1.804375,
 1051: 0.3276114180672269,
 1055: 0.3879253787878787,
 1061: 0.7652068494776828,
 1062: 0.5791176470588235,
 1066: 0.2350694444444444,
 1070: 7.504453125,
 1074: 0.1416964285714285,
 1075: 5.9175,
 1080: 6.594414737654321,
 1082: 3.54,
 1084: 1.3653343461398166,
 1085: 11.880624999999998,
 1086: 7.70125,
 1091: 0.01221875,
 1092: 6.78,
 1093: 11.945,
 1095: 0.974625,
 1096: 9.362916666666669,
 1098: 3.889583333333333,
 1101: 0.4560589015151515,
 1103: 2.8982431881390434,
 1105: 2.493125,
 1111: 0.9206644613368542,
 1114: 3.872853331043957,
 1115: 0.4911770656389839,
 1118: 6.1025,
 1119: 0.824041666666

In [50]:
# Agregamos el precio unitario a cada item
monthly_preds['unit_price'] = monthly_preds['item'].apply(lambda x: item_price[x])
monthly_preds

,item,month,y_real,y_binario,y_croston,y_croston_est,unit_price
0,1002,2023-05-31,0,0.0,4.254629,7.539547,0.009178
1,1002,2023-06-30,0,0.0,31.909721,9.159404,0.009178
2,1002,2023-07-31,0,0.0,32.973379,0.000000,0.009178
3,1002,2023-08-31,0,0.0,32.973379,72.379647,0.009178
4,1002,2023-09-30,0,0.0,31.909721,7.754962,0.009178
...,...,...,...,...,...,...,...
24317,9998,2023-07-31,0,0.0,40.323716,77.085291,0.390132
24318,9998,2023-08-31,2,0.0,40.323716,22.201770,0.390132
24319,9998,2023-09-30,0,0.0,39.022951,39.427280,0.390132
24320,9998,2023-10-31,0,0.0,40.323716,0.765578,0.390132


In [51]:
# Agregamos el precio unitario a cada item
monthly_preds['unit_price'] = monthly_preds['item'].apply(lambda x: item_price[x])
monthly_preds

,item,month,y_real,y_binario,y_croston,y_croston_est,unit_price
0,1002,2023-05-31,0,0.0,4.254629,7.539547,0.009178
1,1002,2023-06-30,0,0.0,31.909721,9.159404,0.009178
2,1002,2023-07-31,0,0.0,32.973379,0.000000,0.009178
3,1002,2023-08-31,0,0.0,32.973379,72.379647,0.009178
4,1002,2023-09-30,0,0.0,31.909721,7.754962,0.009178
...,...,...,...,...,...,...,...
24317,9998,2023-07-31,0,0.0,40.323716,77.085291,0.390132
24318,9998,2023-08-31,2,0.0,40.323716,22.201770,0.390132
24319,9998,2023-09-30,0,0.0,39.022951,39.427280,0.390132
24320,9998,2023-10-31,0,0.0,40.323716,0.765578,0.390132


In [52]:
utility_df = monthly_preds.copy()[['month', 'item', 'y_real', 'y_croston', 'y_croston_est', 'unit_price']]
utility_df

,month,item,y_real,y_croston,y_croston_est,unit_price
0,2023-05-31,1002,0,4.254629,7.539547,0.009178
1,2023-06-30,1002,0,31.909721,9.159404,0.009178
2,2023-07-31,1002,0,32.973379,0.000000,0.009178
3,2023-08-31,1002,0,32.973379,72.379647,0.009178
4,2023-09-30,1002,0,31.909721,7.754962,0.009178
...,...,...,...,...,...,...
24317,2023-07-31,9998,0,40.323716,77.085291,0.390132
24318,2023-08-31,9998,2,40.323716,22.201770,0.390132
24319,2023-09-30,9998,0,39.022951,39.427280,0.390132
24320,2023-10-31,9998,0,40.323716,0.765578,0.390132


In [53]:
# Precio promedio productos
classification_df['unit_price'].mean() 

2.374940656558404

In [54]:
# Definimos un costo fijo de inventario para todos los productos
# Este costo lo definimos como un 30% del precio promedio de todos los productos
STOCK_COST = classification_df['unit_price'].mean() * 0.3
STOCK_COST

0.7124821969675212

In [55]:
# Calculamos los costos por exceso de inventario por cada modelo
def get_stock_cost(row, model):
    """
    Calcula el costo de inventario cuando la cantidad predicha es mayor o igual a la cantidad observada,
    según cada modelo.
    """
    if model == 'croston':
        target = 'y_croston'
    elif model == 'croston_est':
        target = 'y_croston_est'
    else:
        target = 'y_croston'


    if row[target] >= row['y_real']:
        stock_in_excess = (row[target] - row['y_real']) * STOCK_COST
    else:
        stock_in_excess = 0

    return stock_in_excess

for m in ['croston', 'croston_est']:
    utility_df[f'excess_stock_cost_{m}'] = utility_df.apply(lambda x: get_stock_cost(x, m), axis=1)

utility_df

,month,item,y_real,y_croston,y_croston_est,unit_price,excess_stock_cost_croston,excess_stock_cost_croston_est
0,2023-05-31,1002,0,4.254629,7.539547,0.009178,3.031348,5.371793
1,2023-06-30,1002,0,31.909721,9.159404,0.009178,22.735108,6.525912
2,2023-07-31,1002,0,32.973379,0.000000,0.009178,23.492945,0.000000
3,2023-08-31,1002,0,32.973379,72.379647,0.009178,23.492945,51.569210
4,2023-09-30,1002,0,31.909721,7.754962,0.009178,22.735108,5.525272
...,...,...,...,...,...,...,...,...
24317,2023-07-31,9998,0,40.323716,77.085291,0.390132,28.729929,54.921898
24318,2023-08-31,9998,2,40.323716,22.201770,0.390132,27.304965,14.393401
24319,2023-09-30,9998,0,39.022951,39.427280,0.390132,27.803158,28.091235
24320,2023-10-31,9998,0,40.323716,0.765578,0.390132,28.729929,0.545461


In [56]:
# Calculamos los costos por quiebre de stock por cada modelo
def get_stock_out_cost(row, model):
    """
    Calcula el costo de quiebre de stock cuando la cantidad predicha es menor a la cantidad observada,
    según cada modelo. Este costo está dado por el precio de venta del item por la cantidad de ítems no vendidos 
    a causa del quiebre de stock
    """
    if model == 'croston':
        target = 'y_croston'
    elif model == 'croston_est':
        target = 'y_croston_est'
    else:
        target = 'y_croston'


    if row['y_real'] > row[target]:
        stock_out = (row['y_real'] - row[target]) * row['unit_price']
    else:
        stock_out = 0

    return stock_out

for m in ['croston', 'croston_est']:
    utility_df[f'stock_out_cost_{m}'] = utility_df.apply(lambda x: get_stock_out_cost(x, m), axis=1)

utility_df

,month,item,y_real,y_croston,y_croston_est,unit_price,excess_stock_cost_croston,excess_stock_cost_croston_est,stock_out_cost_croston,stock_out_cost_croston_est
0,2023-05-31,1002,0,4.254629,7.539547,0.009178,3.031348,5.371793,0.0,0.0
1,2023-06-30,1002,0,31.909721,9.159404,0.009178,22.735108,6.525912,0.0,0.0
2,2023-07-31,1002,0,32.973379,0.000000,0.009178,23.492945,0.000000,0.0,0.0
3,2023-08-31,1002,0,32.973379,72.379647,0.009178,23.492945,51.569210,0.0,0.0
4,2023-09-30,1002,0,31.909721,7.754962,0.009178,22.735108,5.525272,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
24317,2023-07-31,9998,0,40.323716,77.085291,0.390132,28.729929,54.921898,0.0,0.0
24318,2023-08-31,9998,2,40.323716,22.201770,0.390132,27.304965,14.393401,0.0,0.0
24319,2023-09-30,9998,0,39.022951,39.427280,0.390132,27.803158,28.091235,0.0,0.0
24320,2023-10-31,9998,0,40.323716,0.765578,0.390132,28.729929,0.545461,0.0,0.0


In [57]:
# Calculamos los ingresos por venta de repuesto por cada modelo
def get_income_earned_by_sale(row, model):
    """
    Calcula el ingreso obtenido por la venta de repuesto
    """
    if model == 'croston':
        target = 'y_croston'
    elif model == 'croston_est':
        target = 'y_croston_est'
    else:
        target = 'y_croston'


    if row[target] >= row['y_real']:
        sales = row['y_real'] * row['unit_price']
    else:
        sales = row[target] * row['unit_price']

    return sales

for m in ['croston', 'croston_est']:
    utility_df[f'sales_income_{m}'] = utility_df.apply(lambda x: get_income_earned_by_sale(x, m), axis=1)

utility_df

,month,item,y_real,y_croston,y_croston_est,unit_price,excess_stock_cost_croston,excess_stock_cost_croston_est,stock_out_cost_croston,stock_out_cost_croston_est,sales_income_croston,sales_income_croston_est
0,2023-05-31,1002,0,4.254629,7.539547,0.009178,3.031348,5.371793,0.0,0.0,0.000000,0.000000
1,2023-06-30,1002,0,31.909721,9.159404,0.009178,22.735108,6.525912,0.0,0.0,0.000000,0.000000
2,2023-07-31,1002,0,32.973379,0.000000,0.009178,23.492945,0.000000,0.0,0.0,0.000000,0.000000
3,2023-08-31,1002,0,32.973379,72.379647,0.009178,23.492945,51.569210,0.0,0.0,0.000000,0.000000
4,2023-09-30,1002,0,31.909721,7.754962,0.009178,22.735108,5.525272,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
24317,2023-07-31,9998,0,40.323716,77.085291,0.390132,28.729929,54.921898,0.0,0.0,0.000000,0.000000
24318,2023-08-31,9998,2,40.323716,22.201770,0.390132,27.304965,14.393401,0.0,0.0,0.780264,0.780264
24319,2023-09-30,9998,0,39.022951,39.427280,0.390132,27.803158,28.091235,0.0,0.0,0.000000,0.000000
24320,2023-10-31,9998,0,40.323716,0.765578,0.390132,28.729929,0.545461,0.0,0.0,0.000000,0.000000


In [58]:
# Calculamos la utilidad para cada dato
def get_utility(row, model):
    """
    Calculamos la utilidad para cada dato:
    Suma de ingresos - Suma de costos
    """
    total_incomes = row[f'sales_income_{model}']
    total_costs = row[f'excess_stock_cost_{model}'] + row[f'stock_out_cost_{model}']
    utility = total_incomes - total_costs

    return utility

for m in ['croston', 'croston_est']:
    utility_df[f'utility_{m}'] = utility_df.apply(lambda x: get_utility(x, m), axis=1)

utility_df

,month,item,y_real,y_croston,y_croston_est,unit_price,excess_stock_cost_croston,excess_stock_cost_croston_est,stock_out_cost_croston,stock_out_cost_croston_est,sales_income_croston,sales_income_croston_est,utility_croston,utility_croston_est
0,2023-05-31,1002,0,4.254629,7.539547,0.009178,3.031348,5.371793,0.0,0.0,0.000000,0.000000,-3.031348,-5.371793
1,2023-06-30,1002,0,31.909721,9.159404,0.009178,22.735108,6.525912,0.0,0.0,0.000000,0.000000,-22.735108,-6.525912
2,2023-07-31,1002,0,32.973379,0.000000,0.009178,23.492945,0.000000,0.0,0.0,0.000000,0.000000,-23.492945,0.000000
3,2023-08-31,1002,0,32.973379,72.379647,0.009178,23.492945,51.569210,0.0,0.0,0.000000,0.000000,-23.492945,-51.569210
4,2023-09-30,1002,0,31.909721,7.754962,0.009178,22.735108,5.525272,0.0,0.0,0.000000,0.000000,-22.735108,-5.525272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24317,2023-07-31,9998,0,40.323716,77.085291,0.390132,28.729929,54.921898,0.0,0.0,0.000000,0.000000,-28.729929,-54.921898
24318,2023-08-31,9998,2,40.323716,22.201770,0.390132,27.304965,14.393401,0.0,0.0,0.780264,0.780264,-26.524701,-13.613137
24319,2023-09-30,9998,0,39.022951,39.427280,0.390132,27.803158,28.091235,0.0,0.0,0.000000,0.000000,-27.803158,-28.091235
24320,2023-10-31,9998,0,40.323716,0.765578,0.390132,28.729929,0.545461,0.0,0.0,0.000000,0.000000,-28.729929,-0.545461


In [59]:
# Calculamos la utilidad final para cada modelo
utility = {}
for m in ['croston', 'croston_est']:
    utility[m] = utility_df[f'utility_{m}'].sum()

utility

{'croston': -1235960.1305210628, 'croston_est': -426611.01544252445}

In [67]:
results_df[results_df.item == 1051]['croston_est_mae'].mean()

0.5412473083122072

In [66]:
results_df

,item,demand_type,n_points,bin_reg_mae,croston_mae,croston_est_mae
0,1002,lumpy,1793,0.222841,1.274647,1.348773
1,1003,intermittent,1170,0.008547,0.040301,0.025591
2,1006,lumpy,1947,0.079487,0.382824,0.337329
3,1007,smooth,1,NaN,NaN,NaN
4,1009,smooth,1,NaN,NaN,NaN
...,...,...,...,...,...,...
3812,9988,intermittent,624,0.016000,1.984000,0.361180
3813,9989,smooth,1,NaN,NaN,NaN
3814,9994,intermittent,1800,0.072222,0.143770,0.100775
3815,9997,smooth,1,NaN,NaN,NaN
